# Demo of the Make3D dataset
http://make3d.cs.cornell.edu/data.html#make3d

In [ ]:
import tensorflow as tf
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt

from Make3D import train_pairs, test_pairs

In [ ]:
fig, axis = plt.subplots(5, 2, figsize=(10,20))
plt.tight_layout()
for (rgb, d), (ax1, ax2) in zip(train_pairs[:10], axis):
    ax1.imshow(rgb)
    ax2.imshow(sp.misc.imresize(d, rgb.shape))
    ax1.axis('off')
    ax2.axis('off')
plt.show()